In [ ]:
# 1. FORCE KILL EVERYTHING & WIPE DIRECTORIES
print("🧹 [1/5] Nuclear Wipe of old sessions...")
!pkill -9 dockerd || true
!pkill -9 cloudflared || true
!rm -rf /var/run/docker.sock
!rm -rf /var/run/docker.pid
!rm -rf /etc/apt/sources.list.d/r2u.list

# 2. INSTALL CLEAN BINARIES
print("🏗️ [2/5] Reinstalling Engine...")
!apt-get update -qq > /dev/null
!apt-get install -y docker.io adb -qq > /dev/null

# 3. START ENGINE WITH NO-RESTRICTION FLAG
print("⚡ [3/5] Forcing Motor Start...")
import os, subprocess, time, threading
# We use --iptables=false to stop Google's firewall from blocking Docker internally
get_ipython().system_raw('dockerd --storage-driver=vfs --iptables=false --host=unix:///var/run/docker.sock &')
time.sleep(20)

# 4. LAUNCH ANDROID + VIEWER (Unified Package)
print("📱 [4/5] Booting Android...")
try:
    !docker run -d --privileged --name android -p 5555:5555 redroid/redroid:11.0.0-latest
    !docker run -d --privileged --name viewer --link android -p 7860:8080 -e A_HOST=android -e A_PORT=5555 freddy00/redroid-web
except:
    print("⚠️ Docker busy, attempting recovery...")
    !docker rm -f android viewer || true
    !docker run -d --privileged --name android -p 5555:5555 redroid/redroid:11.0.0-latest
    !docker run -d --privileged --name viewer --link android -p 7860:8080 -e A_HOST=android -e A_PORT=5555 freddy00/redroid-web

# 5. START CLOUDFLARE DIRECT BINARY
print("🔗 [5/5] Creating Link (No Password)...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

def run_tunnel():
    os.system('./cloudflared tunnel --url http://localhost:7860')

threading.Thread(target=run_tunnel, daemon=True).start()
time.sleep(15)
print("\n" + "="*60)
print("✅ SUCCESS! SCROLL DOWN FOR THE .trycloudflare.com LINK")
print("="*60)